In [10]:
import sys
sys.path.append("../..")
from MOF_build.functions.read import read
from MOF_build.functions.normv import normalize_vector
from MOF_build.functions.rotate import rotate
import pandas as pd

In [11]:
#methyl=read.pdb('/Users/chenxili/GitHub/MOF_build/UIO66_OLD/methyl.pdb')
terms=read.pdb('../../MOF_build/lib/PCN222_lib_cut.lib')
terms


,Atom_label,Residue,Res_number,x,y,z,Note
0,O2,CUT,1,0.218,-4.044,0.282,O1-
1,O1,CUT,1,0.374,-3.529,-1.885,O1-
2,C1,CUT,1,0.433,-4.317,-0.923,C
3,O2,CUT,2,1.890,3.152,-1.725,O1-
4,O1,CUT,2,3.459,1.907,-0.740,O1-
5,C1,CUT,2,3.077,2.832,-1.481,C
6,O2,CUT,3,1.890,-2.952,2.048,O1-
7,O1,CUT,3,3.459,-1.514,1.374,O1-
8,C1,CUT,3,3.077,-2.590,1.871,C
9,O2,CUT,4,0.218,2.060,-3.492,O1-


In [12]:
def get_new_cut(cut):
    C1 = cut.loc[2, ["x", "y", "z"]].to_numpy()
    CC = normalize_vector(C1) 
    methyl=read.pdb('D:/OneDrive - KTH/data/code/MOF_build/UIO66_OLD/methyl.pdb')
    methylH1=methyl.loc[1, ["x", "y", "z"]].to_numpy()
    methylH2=methyl.loc[2, ["x", "y", "z"]].to_numpy()
    methylH3=methyl.loc[3, ["x", "y", "z"]].to_numpy()
    methylCH = (methylH1+methylH2+methylH3)/3 #methyl C is 0 0 0 
    p1 = methylCH
    p2 = C1
    methyl_arr = methyl.loc[:, ["x", "y", "z"]].to_numpy()

    q=rotate.calculate_q_rotation_with_vectors(p1,p2)

    new_methyl=rotate.get_rotated_array(methyl_arr,q)+1.5*CC+C1
    df_carbo = cut.iloc[0:3,:]

    methyl.loc[:, ["x", "y", "z"]] = new_methyl
    df_newcut = pd.concat([df_carbo,methyl],ignore_index=True,join='outer')
    return df_newcut

In [13]:

for i in range(1):
    start = 3*i
    cut=terms[start:start+3].reset_index(drop=True)
    init=get_new_cut(cut)
df_cut = init
for i in range(1,8):
    start = 3*i
    cut=terms[start:start+3].reset_index(drop=True)
    df_cut=pd.concat([df_cut,get_new_cut(cut)],join='outer',ignore_index=True)


In [14]:
from MOF_build.functions.output import output
df_left = df_cut[["Atom_label", "Residue", "Res_number", "Note"]]
df_right = df_cut[["x","y","z"]]
df = pd.concat([df_left, df_right], axis=1, join="outer")
df.to_csv("newlibcut.txt", header=None, sep="\t", index=False)
output('newlibcut',True,True,True)